In [ ]:
from grid.robot.locomotion.isaac_locomotion import IsaacLocomotion
locomotionisaacsim_0 = IsaacLocomotion()

In [ ]:
# Activate the robot client

locomotionisaacsim_0.run()

In [ ]:
import time
import h5py
import numpy as np
from grid.utils.types import Velocity
from grid.utils import log

# -----------------------------------------------------------------------------
# Initialize lists for storing captured image data and timestamps.
# - The RGB images and depth images are stored in separate lists.
# - Timestamps are recorded to associate each capture with a time.
# -----------------------------------------------------------------------------
rgb_images_list = []    # Will hold raw RGB image data for HDF5 saving.
depth_images_list = []  # Will hold raw depth image data for HDF5 saving.
timestamps_list = []    # Will record the timestamp for each image capture.

# -----------------------------------------------------------------------------
# First Loop: Rotate in Place and Capture Images for 5 Seconds
# -----------------------------------------------------------------------------
start_time = time.time()
while time.time() - start_time < 5:
    # Command the robot to rotate in place.
    # The first argument is linear velocity (set to zero) and the second is angular velocity.
    locomotionisaacsim_0.moveByVelocity(Velocity(0, 0, 0), Velocity(0, 0, 2))

    # Retrieve images from the RGB and depth cameras.
    rgb_img = locomotionisaacsim_0.getImage()
    depth_img = locomotionisaacsim_0.getImage("camera_depth_0")
    current_time = time.time()  # Capture the current time for timestamping.

    # ------------------------------
    # Process and Log the RGB Image
    # ------------------------------
    if rgb_img.data is not None:
        # Log the RGB image for visualization.
        log("rgb_img", rgb_img)
        # Save the raw RGB image data for later HDF5 storage.
        rgb_images_list.append(rgb_img.data)
    else:
        print("Did not receive RGB data")

    # ------------------------------
    # Process and Log the Depth Image
    # ------------------------------
    if depth_img.data is not None:
        # Save the raw depth image data for HDF5 storage.
        depth_images_list.append(depth_img.data)
        # For visualization only: scale the depth image data.
        depth_img.data = depth_img.data * 255
        log("depth_img", depth_img)
    else:
        print("Did not receive depth data")

    # Record the timestamp for this capture.
    timestamps_list.append(current_time)

    # Sleep briefly to control the capture rate (0.2 seconds between captures).
    time.sleep(0.2)

# -----------------------------------------------------------------------------
# Second Loop: Move Forward and Capture Images for 10 Seconds
# -----------------------------------------------------------------------------
start_time = time.time()
while time.time() - start_time < 10:
    # Command the robot to move forward.
    # Linear velocity is set to 1 while angular velocity remains zero.
    locomotionisaacsim_0.moveByVelocity(Velocity(1, 0, 0), Velocity(0, 0, 0))

    # Retrieve images from the RGB and depth cameras.
    rgb_img = locomotionisaacsim_0.getImage()
    depth_img = locomotionisaacsim_0.getImage("camera_depth_0")
    current_time = time.time()  # Record the current time.

    # ------------------------------
    # Process and Log the RGB Image
    # ------------------------------
    if rgb_img.data is not None:
        # Append the raw RGB image data to our list for HDF5 saving.
        rgb_images_list.append(rgb_img.data)
        # Log the RGB image for display.
        log("rgb_img", rgb_img)
    else:
        print("Did not receive RGB data")

    # ------------------------------
    # Process and Log the Depth Image
    # ------------------------------
    if depth_img.data is not None:
        # Save the raw depth image data for HDF5 storage.
        depth_images_list.append(depth_img.data)
        # For visualization only: scale the depth image data.
        depth_img.data = depth_img.data * 255
        log("depth_img", depth_img)
    else:
        print("Did not receive depth data")

    # Save the timestamp for this image capture.
    timestamps_list.append(current_time)

    # Sleep briefly to maintain the capture rate.
    time.sleep(0.2)

# -----------------------------------------------------------------------------
# Stop the Robot's Movement
# -----------------------------------------------------------------------------
# Send zero velocities to halt any movement.
locomotionisaacsim_0.moveByVelocity(Velocity(0, 0, 0), Velocity(0, 0, 0))
print("Done!")


In [ ]:
import os

# -----------------------------------------------------------------------------
# Convert the Captured Data to NumPy Arrays
# -----------------------------------------------------------------------------
# This conversion is done so that the data can be easily stored in HDF5.
# It assumes that all images have consistent dimensions.
rgb_images_np = np.array(rgb_images_list)
depth_images_np = np.array(depth_images_list)
timestamps_np = np.array(timestamps_list)

# Access the path of the session cloud storage
from grid import GRID_USER_SESSION_BLOB_DIR

hdf5_file_path = os.path.join(GRID_USER_SESSION_BLOB_DIR, "data_log.h5")
# -----------------------------------------------------------------------------
# Save the Data to an HDF5 File
# -----------------------------------------------------------------------------
# The HDF5 file will contain:
# - 'rgb_images': the raw RGB image data.
# - 'depth_images': the raw depth image data (without the 255 multiplier).
# - 'timestamps': the timestamps for each image capture.
with h5py.File(hdf5_file_path, "w") as h5f:
    h5f.create_dataset("rgb_images", data=rgb_images_np, compression="gzip")
    h5f.create_dataset("depth_images", data=depth_images_np, compression="gzip")
    h5f.create_dataset("timestamps", data=timestamps_np, compression="gzip")

print("Images and timestamps have been saved to 'data_log.h5'")
